In [109]:
import os 
import pandas as pd

import plotly.express as px
from IPython.display import HTML
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


ROOTDIR = os.path.dirname(os.getcwd())
DATADIR = os.path.join(ROOTDIR, 'data')


def merch(df, merch):
    return df[df.merch.str.contains(merch, na=False)]

def cat(df, cat):
    return df[df.cat.str.contains(cat, na=False)]

def month(df, ym):
    return df[df.ym.eq(ym)]

In [112]:
df = pd.read_parquet(os.path.join(DATADIR, 'clean.parquet'))

jul = month(df, '2020-07')
aug = month(df, '2020-08')

In [117]:
cat(aug, 'groc').amount.sum()

-635.1399999999999

In [106]:
merch(month(df, '2020-08'), 'tfl')

,date,desc,cat,amount,merch,note,id,split,ym
2348,2020-08-01 00:02:13,tfl travel charge tfl.gov.uk/cp gbr,transport,-3.0,tfl travel charge,None,tx_00009xf2uoxaf4qkaqd1d3,None,2020-08-01
2377,2020-08-08 00:01:32,tfl travel charge tfl.gov.uk/cp gbr,transport,-3.0,tfl travel charge,None,tx_00009xtyrzhg1rwlypkaij,None,2020-08-01
2380,2020-08-08 08:25:38,tfl cycle hire london gbr,transport,-4.0,tfl santander cycles,None,tx_00009xuhqpza1y3ouw78wp,None,2020-08-01
2397,2020-08-14 00:01:25,tfl travel charge tfl.gov.uk/cp gbr,transport,-3.0,tfl travel charge,None,tx_00009y5zw1gqukvhjexeyf,None,2020-08-01
2410,2020-08-17 12:44:06,tfl travel charge tfl.gov.uk/cp gbr,transport,-1.5,tfl travel charge,None,tx_00009ydj7hnwa2oxva52v0,None,2020-08-01
2415,2020-08-20 00:01:44,tfl travel charge tfl.gov.uk/cp gbr,transport,-3.0,tfl travel charge,None,tx_00009yiqczfzpigy5arsmp,None,2020-08-01


In [118]:
df.cat.value_counts()

transport        497
eating out       299
general          204
groceries        192
shopping         108
bills             78
expenses          65
holidays          58
entertainment     53
personal care     52
finances          18
health             4
subscriptions      4
transfer           4
fees               1
income             1
Name: cat, dtype: int64

In [132]:
def make_parent_cats(df):

    mains = 'income bills health groceries holidays'.split()
    
    df['pcat'] = 'other'
    return df.apply(lambda row: row.pcat = 'he' if row.amount.eq(200) else row, axis=1)

make_parent_cats(df)

SyntaxError: lambda cannot contain assignment (<ipython-input-132-120d53d72355>, line 6)

# Resources

- Using Plotly for Jupyter Lab and Fastpages ([link](https://colab.research.google.com/github/binnisb/blog/blob/master/_notebooks/2020-04-02-Plotly-in-lab.ipynb#scrollTo=TWW5ogvQPEk1))